In [ ]:
from typing import   List
from dotenv import load_dotenv
from langchain.agents import Tool

from langchain_openai import ChatOpenAI
import os
import re
from html import unescape
from openai_assistant_tools import TradingviewWrapper
from langchain.agents import tool
import json
import os


from langchain.agents import Tool
from openai_assistant_tools import GoogleSerperAPIWrapper
from openai_assistant_tools import MyAPIChain

import openai_assistant_api_docs

import openai_assistant_api_docs
from langchain.agents import  AgentExecutor
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
import asyncio

# from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain_core.runnables import Runnable


load_dotenv(dotenv_path=".env")



def getTools()->:
# agent_cb_manager = AsyncCallbackManager([agent_cb_handler])
    llm = ChatOpenAI(
        model="gpt-3.5-turbo-1106",
        verbose=True,
    )
    headers = {
        "Accepts": "application/json",
        "X-CMC_PRO_API_KEY": os.getenv("CMC_API_KEY"),
    }
    cmc_last_quote_api = MyAPIChain.from_llm_and_api_docs(
        llm=llm,
        api_docs=openai_assistant_api_docs.cmc_quote_lastest_api_doc,
        headers=headers,
        limit_to_domains=["https://pro-api.coinmarketcap.com"],
        verbose=True,
    )

    @tool
    def getTokenMetadata(symbol: str) -> str:
        """
        Useful when you need get the metadata of a token.
        """
        url = (
            f"https://pro-api.coinmarketcap.com/v2/cryptocurrency/info?symbol={symbol}"
        )
        response = requests.get(url, headers=headers)
        return json.dumps(response.json())

    tradingview = TradingviewWrapper(llm=llm)

    @tool
    def search(query: str, search_type: str = None) -> str:
        """Search for a webpage with Google based on the query.
        Set the optional search_type (str) parameter to specify whether to search news (search_type='news') or web pages (search_type=None).
        """
        if search_type == "news":
            newsSearch = GoogleSerperAPIWrapper(type=search_type, tbs="qdr:h")
            return json.dumps(
                [
                    {
                        "title": r["title"],
                        "link": r["link"],
                        "snippet": r["snippet"],
                        "imageUrl": r["imageUrl"],
                    }
                    for r in newsSearch.results(query=query)["news"]
                ]
            )
        else:
            searchWebpage = GoogleSerperAPIWrapper()
            return json.dumps(
                [
                    {
                        "title": r["title"],
                        "link": r["link"],
                        "snippet": r["snippet"],
                    }
                    for r in searchWebpage.results(query=query)["organic"]
                ]
            )

    def remove_html_tags(text):
        """Remove html tags from a string"""
        clean = re.compile("<.*?>")
        text = re.sub(clean, "", text)  # Remove HTML tags
        text = unescape(text)  # Unescape HTML entities
        return text

    from pyppeteer import launch

    async def fetch_page(url):
        browser = await launch()
        page = await browser.newPage()
        await page.goto(url)
        html_content = await page.content()
        await browser.close()
        return html_content

    @tool
    async def get_contents(links: List[str]):
        """Get the contents of a webpage.
        The links passed in should be a list of links returned from `search`.
        """
        req_tasks = []
        results = []
        for url in links:
            req_tasks.append(fetch_page(url=url))
            results.append(
                {
                    "url": url,
                }
            )
        contents = await asyncio.gather(*req_tasks)
        extract_task = []
        for _content in contents:
            no_html = remove_html_tags(_content)
            prompt = ChatPromptTemplate.from_template(
                """I have a piece of text that I need you to help summarize, but please ensure that the summary does not exceed 100 words. Here is the text that needs to be summarized: {input}."""
            )
            model = ChatOpenAI(model="gpt-3.5-turbo-1106", verbose=True)
            output_parser = StrOutputParser()
            chain = prompt | model | output_parser
            task = chain.with_config({"verbose": True}).ainvoke({"input": no_html})
            extract_task.append(task)
        _extracts = await asyncio.gather(*extract_task)
        for i in range(len(results)):
            results[i]["extract"] = _extracts[i]
        return json.dumps(results) if len(results) > 0 else f"There is no any result"

    # from langchain_community.tools.arxiv.tool import ArxivQueryRun
    from arxiv_wrapper import ArxivAPIWrapper

    # arxiv = ArxivQueryRun()

    @tool
    def arxiv_search(query: str):
        """A wrapper around Arxiv.org
        Useful for when you need to answer questions about Physics, Mathematics,
        Computer Science, Quantitative Biology, Quantitative Finance, Statistics,
        Electrical Engineering, and Economics
        from scientific articles on arxiv.org.
        Input should be a search query."""
        api_wrapper = ArxivAPIWrapper(doc_content_chars_max=10000)
        return api_wrapper.run(query=query)

    @tool
    def arxiv_load(entry_id: str):
        """Useful for when your need to know the content of some paper on Arxiv.org.
        Input should be the entry_id return from `arxiv_search`."""
        api_wrapper = ArxivAPIWrapper(doc_content_chars_max=10000)
        return api_wrapper.load(query=entry_id)

    import requests
    from bs4 import BeautifulSoup

    @tool
    def getHTMLFromURL(url: str) -> str:
        """useful when you need get the HTML of URL. The input to this should be URL."""
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        return soup.prettify()

    @tool
    async def getHTMLFromURLs(urls: list[str]) -> str:
        """useful when you need get the HTML of URLs. The input to this should be URL list."""
        req_tasks = []
        for url in urls:
            req_tasks.append(fetch_page(url=url))
        contents = await asyncio.gather(*req_tasks)
        result = ""
        for c in contents:
            soup = BeautifulSoup(c, "html.parser")
            result += "\n" + remove_html_tags(soup.prettify())
        return result

    tools = [
        search,
        getHTMLFromURL,
        getHTMLFromURLs,
        Tool(
            name="CryptocurrencyLatestQuote",
            func=cmc_last_quote_api.run,
            description="""useful when you need get a cryptocurrency's latest quote. The input to this should be a single cryptocurrency's symbol.""",
            coroutine=cmc_last_quote_api.arun,
        ),
        getTokenMetadata,
        Tool(
            name="BuyOrSellSignal",
            func=tradingview.buySellSignal,
            description="""Useful when you need to know buy and sell signals for a cryptocurrency. The input to this should be a cryptocurrency's symbol.""",
            coroutine=tradingview.abuySellSignal,
        ),
        arxiv_search,
        arxiv_load,
    ]
    from zero_scope_agent import zero_scope_tools

    tools += zero_scope_tools
    return tools


def create_agent_executor(llm_agent: Runnable) -> AgentExecutor:

    tools=getTools() 
    
    from langchain.tools.render import render_text_description
    from langchain_core.runnables import RunnablePassthrough
    from langchain.agents.format_scratchpad import format_log_to_str
    from langchain.agents.output_parsers import ReActSingleInputOutputParser
    from langchain import hub

    react_prompt = hub.pull("hwchase17/react-chat")
    react_prompt = react_prompt.partial(
        tools=render_text_description(list(tools)),
        tool_names=", ".join([t.name for t in tools]),
    )
    agent = (
        # {
        #     "input": lambda x: x["input"],
        #     "agent_scratchpad": lambda x: format_log_to_str(
        #         x["intermediate_steps"]
        #     ),
        #     "chat_history": lambda x: x["chat_history"],
        # }
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        )
        | react_prompt
        | llm_agent.bind(stop=["\nObservation"])
        | ReActSingleInputOutputParser()
    )

    executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
    ).with_config({"run_name": "Eddie's Assistant Agent"})
    return executor


from langchain_community.chat_models import ChatPerplexity

llm_agent = ChatPerplexity(
    model="sonar-medium-chat", temperature=0.9, verbose=True, streaming=True
)

agent_executor = create_agent_executor(llm_agent=llm_agent)